# import stuff.

In [1]:
from os import environ
from dotenv import load_dotenv
import numpy as np
import pandas as pd
from riotwatcher import LolWatcher, ApiError

load_dotenv("../")

RIOT_API = environ["RIOT_API"]
lol_watcher = LolWatcher(RIOT_API)
my_region = "na1"
NA = "na1"

# initialize stuff

In [2]:
yogurtsauce = pd.DataFrame.from_dict(
    lol_watcher.summoner.by_name(NA, "yogurtsauce"), orient="index"
).T

ranked = lol_watcher.league.by_summoner(NA, yogurtsauce.id[0])
versions = lol_watcher.data_dragon.versions_for_region(NA)
champions_version = versions["n"]["champion"]
summoner_spells_version = versions["n"]["summoner"]
items_version = versions["n"]["item"]
current_champ_list = lol_watcher.data_dragon.champions(champions_version)
my_matches = lol_watcher.match.matchlist_by_puuid(NA, yogurtsauce.puuid[0])
current_items = lol_watcher.data_dragon.items(items_version)
mastery_version = versions["n"]["mastery"]

# league

In [3]:
class League:
    def __init__(self, region) -> None:
        self.versions = self.Versions(region)
        self.champions = self.get_champions(region)

    class Versions:
        def __init__(self, region) -> None:
            self.versions_ = lol_watcher.data_dragon.versions_for_region(region)
            self.versions_item = self.versions_["n"]["item"]
            self.versions_rune = self.versions_["n"]["rune"]
            self.versions_mastery = self.versions_["n"]["mastery"]
            self.versions_summoner = self.versions_["n"]["summoner"]
            self.versions_champion = self.versions_["n"]["champion"]

    def get_champions(self, region):
        champions = (
            pd.DataFrame.from_dict(
                lol_watcher.data_dragon.champions(
                    self.Versions(region).versions_champion
                )
            )
            .reset_index()
            .rename(columns={"index": "champion"})
        )

        champions = (
            pd.concat([champions, pd.json_normalize(champions.data)], axis=1)
            .drop(columns=["type", "format", "version", "data", "name", "id"])
            .rename(
                {
                    "key": "champion_id",
                    "partype": "resource",
                    # info
                    "info.attack": "attack",
                    "info.defense": "defense",
                    "info.magic": "magic",
                    "info.difficulty": "difficulty",
                    # stats
                    # hp
                    "stats.hp": "hp",
                    "stats.hpperlevel": "hp_level",
                    "stats.hpregen": "hp_regen",
                    "stats.hpregenperlevel": "hpregen_level",
                    # mp
                    "stats.mp": "mp",
                    "stats.mpperlevel": "mp_level",
                    "stats.mpregen": "mp_regen",
                    "stats.mpregenperlevel": "mpregen_level",
                    # ad
                    "stats.attackdamage": "attack",
                    "stats.attackdamageperlevel": "attack_level",
                    # crit
                    "stats.crit": "crit",
                    "stats.critperlevel": "crit_level",
                    # armour
                    "stats.armor": "armor",
                    "stats.armorperlevel": "armor_level",
                    # attack speed
                    "stats.attackspeed": "atk_spd",
                    "stats.attackspeedperlevel": "atk_spd_level",
                    # ms
                    "stats.movespeed": "move_spd",
                    # attack range
                    "stats.attackrange": "atk_range",
                    # spellblock
                    "stats.spellblock": "spellblock",
                    "stats.spellblockperlevel": "spellblock_level",
                },
                axis=1,
            )
        )

        champions.insert(0, "champion_id", champions.pop("champion_id"))
        
        champions[["primary_role", "secondary_role"]] = pd.DataFrame(
            champions.tags.tolist(), index=champions.index
        )
        champions = champions.drop(columns="tags")
        
        # champions.primary_role = champions.primary_role.astype("category")
        # champions.secondary_role = champions.secondary_role.astype("category")
        champions.resource = champions.resource.astype("string")
        champions.champion_id = champions.champion_id.astype("int")

        return champions


League("na1").champions.head(1)

,champion_id,champion,title,blurb,resource,attack,defense,magic,difficulty,image.full,...,mp_regen,mpregen_level,crit,crit_level,attack,attack_level,atk_spd_level,atk_spd,primary_role,secondary_role
0,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,Blood Well,8,4,3,4,Aatrox.png,...,0.0,0.0,0,0,60,5.0,2.5,0.651,Fighter,Tank


# summoner

In [7]:
class Summoner:
    def __init__(self, region, summoner_name):
        self.region = region
        self.name = summoner_name
        self.data = pd.DataFrame.from_dict(
            lol_watcher.summoner.by_name(self.region, self.name), orient="index"
        ).T
        self.id = self.data.id[0]
        self.account_id = self.data.accountId[0]
        self.puuid = self.data.puuid[0]
        self.profile_icon_id = self.data.profileIconId[0]
        self.revision_date = self.data.revisionDate[0]
        self.level = self.data.summonerLevel[0]
        self.matches_ = lol_watcher.match.matchlist_by_puuid(self.region, self.puuid)


yogurtsauce = Summoner("na1", "yogurtsauce")

# champions

In [8]:
champions = (
    pd.DataFrame.from_dict(current_champ_list)
    .reset_index()
    .rename(columns={"index": "champion"})
)

champions = (
    pd.concat([champions, pd.json_normalize(champions.data)], axis=1)
    .drop(columns=["type", "format", "version", "data", "name", "id"])
    .rename(
        {
            "key": "champion_id",
            "partype": "resource",
            # info
            "info.attack": "attack",
            "info.defense": "defense",
            "info.magic": "magic",
            "info.difficulty": "difficulty",
            # stats
            # hp
            "stats.hp": "hp",
            "stats.hpperlevel": "hp_level",
            "stats.hpregen": "hp_regen",
            "stats.hpregenperlevel": "hpregen_level",
            # mp
            "stats.mp": "mp",
            "stats.mpperlevel": "mp_level",
            "stats.mpregen": "mp_regen",
            "stats.mpregenperlevel": "mpregen_level",
            # ad
            "stats.attackdamage": "attack",
            "stats.attackdamageperlevel": "attack_level",
            # crit
            "stats.crit": "crit",
            "stats.critperlevel": "crit_level",
            # armour
            "stats.armor": "armor",
            "stats.armorperlevel": "armor_level",
            # attack speed
            "stats.attackspeed": "atk_spd",
            "stats.attackspeedperlevel": "atk_spd_level",
            # ms
            "stats.movespeed": "move_spd",
            # attack range
            "stats.attackrange": "atk_range",
            # spellblock
            "stats.spellblock": "spellblock",
            "stats.spellblockperlevel": "spellblock_level",
        },
        axis=1,
    )
)

champions.insert(0, "champion_id", champions.pop("champion_id"))
champions[["primary_role", "secondary_role"]] = pd.DataFrame(
    champions.tags.tolist(), index=champions.index
)
champions = champions.drop(columns="tags")
# champions.primary_role = champions.primary_role.astype("category")
# champions.secondary_role = champions.secondary_role.astype("category")
champions.resource = champions.resource.astype("string")
champions.champion_id = champions.champion_id.astype("int")

In [9]:
champions.head(1)

,champion_id,champion,title,blurb,resource,attack,defense,magic,difficulty,image.full,...,mp_regen,mpregen_level,crit,crit_level,attack,attack_level,atk_spd_level,atk_spd,primary_role,secondary_role
0,266,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,Blood Well,8,4,3,4,Aatrox.png,...,0.0,0.0,0,0,60,5.0,2.5,0.651,Fighter,Tank


In [10]:
champions[(champions["champion"] == "Aatrox")].resource[0]

'Blood Well'

# match history

In [11]:
def match_history(username, region, n_games=5):
    i = 0
    j = 0
    user = Summoner(region, username)
    matches = pd.DataFrame()

    while j < n_games:
        try:
            last_match = user.matches_[j]
            match_details = lol_watcher.match.by_id(region, last_match)
            match_players = match_details["info"]["participants"]
            details = []

            for player in match_players:
                dict = {}
                dict["ign"] = player["summonerName"]
                dict["level"] = player["summonerLevel"]
                dict["champion_id"] = player["championId"]
                dict["champion"] = player["championName"]
                dict[
                    "mastery_points"
                ] = lol_watcher.champion_mastery.by_summoner_by_champion(
                    region, player["summonerId"], player["championId"]
                )[
                    "championPoints"
                ]
                dict["team"] = player["teamId"]
                dict["win"] = player["win"]

                details.append(dict)

            details = pd.DataFrame.from_dict(details)
            details = details.merge(
                League("na1").champions[
                    [
                        "champion",
                        "attack",
                        "defense",
                        "magic",
                        "difficulty",
                        "resource",
                    ]
                ]
            )

            details.team = details.team.astype("category")

            matches = pd.concat([matches, details], ignore_index=True)

            i += 1
            j += 1
        except:
            j += 1

    return matches


test = match_history("yogurtsauce", "na1", 2)

In [12]:
test

,ign,level,champion_id,champion,mastery_points,team,win,attack,attack,defense,magic,difficulty,resource
0,furrylover616,347,10,Kayle,2841,100,False,6,50,6,7,7,Mana
1,yogurtsauce,515,895,Nilah,2091,100,False,8,58,4,4,10,Mana
2,Axian Superstar,245,126,Jayce,9144,100,False,8,57,4,3,7,Mana
3,Baby Wsup,558,498,Xayah,218700,100,False,10,60,6,1,5,Mana
4,Mołson,607,240,Kled,49829,100,False,8,65,2,2,7,Courage
5,FIddlestankyleg,52,245,Ekko,4873,200,True,5,58,3,7,8,Mana
6,W1lls,491,555,Pyke,32678,200,True,9,62,3,1,7,Mana
7,lolikiller100,268,35,Shaco,26814,200,True,8,63,4,6,9,Mana
8,BomberJumpLA,395,17,Teemo,99062,200,True,5,54,3,7,6,Mana
9,Syrup04,174,888,Renata,10375,200,True,2,49,6,9,8,Mana


In [13]:
last_match = lol_watcher.match.by_id("na1", yogurtsauce.matches_[0])

In [14]:
temp_df = pd.DataFrame()

In [15]:
NUMBER_OF_ROWS = range(10)

red = pd.DataFrame()
blue = pd.DataFrame()

champion_columns = ["champ1", "champ2", "champ3", "champ4", "champ5"]
level_columns = ["level1", "level2", "level3", "level4", "level5"]
mastery_columns = ["mastery_level1", "mastery_level2", "mastery_level3", "mastery_level4", "mastery_level5"]
match_details = last_match["info"]["participants"]
resource_columns = ["resource1", "resource2", "resource3", "resource4", "resource5"]

for x in NUMBER_OF_ROWS:
    if x < 5:
        red.insert(x, champion_columns[x], [match_details[x]["championName"]])
        red.insert(x, level_columns[x], [match_details[x]["summonerLevel"]])
        red.insert(x, mastery_columns[x], [lol_watcher.champion_mastery.by_summoner_by_champion("na1", match_details[x]["summonerId"], match_details[x]["championId"])["championPoints"]])
        red.insert(x, resource_columns[x], [champions[(champions["champion"] == match_details[x]["championName"])].resource])
    else:
        blue.insert(x - 5, champion_columns[x - 5], [match_details[x]["championName"]])
        blue.insert(x - 5, level_columns[x - 5], [match_details[x]["summonerLevel"]])
        blue.insert(x - 5, mastery_columns[x - 5], [lol_watcher.champion_mastery.by_summoner_by_champion("na1", match_details[x]["summonerId"], match_details[x]["championId"])["championPoints"]])
        
red = red.reindex(sorted(red.columns), axis=1)
blue = blue.reindex(sorted(blue.columns), axis=1)
        
red.insert(0, "team", match_details[0]["teamId"])
red.insert(0, "win", match_details[0]["win"])

blue.insert(0, "team", match_details[5]["teamId"])
blue.insert(0, "win", match_details[5]["win"])

In [16]:
list(red.columns)

['win',
 'team',
 'champ1',
 'champ2',
 'champ3',
 'champ4',
 'champ5',
 'level1',
 'level2',
 'level3',
 'level4',
 'level5',
 'mastery_level1',
 'mastery_level2',
 'mastery_level3',
 'mastery_level4',
 'mastery_level5',
 'resource1',
 'resource2',
 'resource3',
 'resource4',
 'resource5']

In [22]:
champions[(champions["champion"] == "Aatrox")].resource[0]

'Blood Well'

In [17]:
red.resource3

0    48    Mana
Name: resource, dtype: string
Name: resource3, dtype: object

In [31]:
red.resource1.astype("str")

0    58    Mana\nName: resource, dtype: string
Name: resource1, dtype: object

In [ ]:
red.resource1

0    58    Mana
Name: resource, dtype: object
Name: resource1, dtype: object

In [ ]:
blue

,win,team,champ1,champ2,champ3,champ4,champ5,level1,level2,level3,level4,level5,mastery_level1,mastery_level2,mastery_level3,mastery_level4,mastery_level5
0,True,200,Ekko,Pyke,Shaco,Teemo,Renata,52,491,268,395,174,4873,32678,26814,99062,10375


In [19]:
aram_matches = pd.DataFrame()

aram_matches = pd.concat([aram_matches, red, blue])
aram_matches

,win,team,champ1,champ2,champ3,champ4,champ5,level1,level2,level3,...,mastery_level1,mastery_level2,mastery_level3,mastery_level4,mastery_level5,resource1,resource2,resource3,resource4,resource5
0,False,100,Kayle,Nilah,Jayce,Xayah,Kled,347,515,245,...,2841,2091,9144,218700,49829,"58 Mana Name: resource, dtype: string","87 Mana Name: resource, dtype: string","48 Mana Name: resource, dtype: string","149 Mana Name: resource, dtype: string","63 Courage Name: resource, dtype: string"
0,True,200,Ekko,Pyke,Shaco,Teemo,Renata,52,491,268,...,4873,32678,26814,99062,10375,NaN,NaN,NaN,NaN,NaN


In [ ]:
match_df = pd.json_normalize(last_match["info"])

pd.json_normalize(match_df.participants[0])

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,...,challenges.visionScorePerMinute,challenges.wardTakedowns,challenges.wardTakedownsBefore20M,challenges.wardsGuarded,perks.statPerks.defense,perks.statPerks.flex,perks.statPerks.offense,perks.styles,challenges.highestChampionDamage,challenges.highestCrowdControlScore
0,0,0,39,0,0,0,1,27989,18,10,...,0,2,0,0,5002,5008,5005,"[{'description': 'primaryStyle', 'selections':...",NaN,NaN
1,0,0,36,0,0,0,1,28282,18,895,...,0,1,1,0,5002,5008,5005,"[{'description': 'primaryStyle', 'selections':...",NaN,NaN
2,0,0,34,0,0,0,1,29808,18,126,...,0,5,4,0,5002,5008,5008,"[{'description': 'primaryStyle', 'selections':...",1.0,NaN
3,0,0,36,0,0,0,1,27433,18,498,...,0,2,2,0,5002,5008,5005,"[{'description': 'primaryStyle', 'selections':...",NaN,NaN
4,0,0,33,0,0,0,0,30664,18,240,...,0,4,2,0,5002,5008,5005,"[{'description': 'primaryStyle', 'selections':...",NaN,NaN
5,0,0,45,0,0,0,0,29935,18,245,...,0,0,0,0,5002,5008,5008,"[{'description': 'primaryStyle', 'selections':...",NaN,NaN
6,0,0,36,0,0,0,0,28136,18,555,...,0,0,0,0,5001,5008,5007,"[{'description': 'primaryStyle', 'selections':...",NaN,NaN
7,0,0,21,0,0,0,0,28715,18,35,...,0,0,0,0,5002,5008,5008,"[{'description': 'primaryStyle', 'selections':...",NaN,NaN
8,0,1,50,0,0,0,0,27971,18,17,...,0,0,0,0,5002,5008,5007,"[{'description': 'primaryStyle', 'selections':...",NaN,1.0
9,0,0,72,0,0,0,0,32255,18,888,...,0,0,0,0,5002,5008,5007,"[{'description': 'primaryStyle', 'selections':...",NaN,NaN


In [ ]:
wins = test.where(test.win == True)
wins = wins[~wins["ign"].isnull()].reset_index(drop=True)

losses = test.where(test.win == False)
losses = losses[~losses["ign"].isnull()].reset_index(drop=True)

In [ ]:
wins["champion1"] = wins["champion"].str.split().str[0][0]
wins["champion2"] = wins["champion"].str.split().str[0][1]
wins["champion3"] = wins["champion"].str.split().str[0][2]
wins["champion4"] = wins["champion"].str.split().str[0][3]
wins["champion5"] = wins["champion"].str.split().str[0][4]

In [ ]:
wins

,ign,level,champion_id,champion,mastery_points,team,win,attack,attack,defense,magic,difficulty,resource,champion1,champion2,champion3,champion4,champion5
0,FIddlestankyleg,52.0,245.0,Ekko,4873.0,200,True,5.0,58.0,3.0,7.0,8.0,Mana,Ekko,Pyke,Shaco,Teemo,Renata
1,W1lls,491.0,555.0,Pyke,32678.0,200,True,9.0,62.0,3.0,1.0,7.0,Mana,Ekko,Pyke,Shaco,Teemo,Renata
2,lolikiller100,268.0,35.0,Shaco,26814.0,200,True,8.0,63.0,4.0,6.0,9.0,Mana,Ekko,Pyke,Shaco,Teemo,Renata
3,BomberJumpLA,395.0,17.0,Teemo,99062.0,200,True,5.0,54.0,3.0,7.0,6.0,Mana,Ekko,Pyke,Shaco,Teemo,Renata
4,Syrup04,174.0,888.0,Renata,10375.0,200,True,2.0,49.0,6.0,9.0,8.0,Mana,Ekko,Pyke,Shaco,Teemo,Renata
5,Mz Piggy,472.0,119.0,Draven,129650.0,200,True,9.0,62.0,3.0,1.0,8.0,Mana,Ekko,Pyke,Shaco,Teemo,Renata
6,LBJ,584.0,59.0,JarvanIV,39555.0,200,True,6.0,64.0,8.0,3.0,5.0,Mana,Ekko,Pyke,Shaco,Teemo,Renata
7,purple flowr,403.0,36.0,DrMundo,22616.0,200,True,5.0,61.0,7.0,6.0,5.0,None,Ekko,Pyke,Shaco,Teemo,Renata
8,yogurtsauce,514.0,245.0,Ekko,10665.0,200,True,5.0,58.0,3.0,7.0,8.0,Mana,Ekko,Pyke,Shaco,Teemo,Renata
9,Salem,716.0,43.0,Karma,68854.0,200,True,1.0,51.0,7.0,8.0,5.0,Mana,Ekko,Pyke,Shaco,Teemo,Renata


In [ ]:
wins["champion"].str.split().str[0][0]

'Ekko'

In [ ]:
wins

,ign,level,champion_id,champion,mastery_points,team,win,attack,attack,defense,magic,difficulty,resource,champion1,champion2
5,FIddlestankyleg,52.0,245.0,Ekko,4873.0,200,True,5.0,58.0,3.0,7.0,8.0,Mana,Ekko,NaN
6,W1lls,491.0,555.0,Pyke,32678.0,200,True,9.0,62.0,3.0,1.0,7.0,Mana,Pyke,NaN
7,lolikiller100,268.0,35.0,Shaco,26814.0,200,True,8.0,63.0,4.0,6.0,9.0,Mana,Shaco,NaN
8,BomberJumpLA,395.0,17.0,Teemo,99062.0,200,True,5.0,54.0,3.0,7.0,6.0,Mana,Teemo,NaN
9,Syrup04,174.0,888.0,Renata,10375.0,200,True,2.0,49.0,6.0,9.0,8.0,Mana,Renata,NaN
15,Mz Piggy,472.0,119.0,Draven,129650.0,200,True,9.0,62.0,3.0,1.0,8.0,Mana,Draven,NaN
16,LBJ,584.0,59.0,JarvanIV,39555.0,200,True,6.0,64.0,8.0,3.0,5.0,Mana,JarvanIV,NaN
17,purple flowr,403.0,36.0,DrMundo,22616.0,200,True,5.0,61.0,7.0,6.0,5.0,None,DrMundo,NaN
18,yogurtsauce,514.0,245.0,Ekko,10665.0,200,True,5.0,58.0,3.0,7.0,8.0,Mana,Ekko,NaN
19,Salem,716.0,43.0,Karma,68854.0,200,True,1.0,51.0,7.0,8.0,5.0,Mana,Karma,NaN
